En este tutorial se hace uso del tool-calling de los modelos. En este tutorial tambien se presenta como el few-shot prompting para mejorar el funcionamiento del modelo.

# The Schema
Primero se describe que información se espera extraer del texto.

Se usa Pydantic para definir el schema de información para extraer sobre un persona:

In [2]:
from typing import Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

Hay dos buenas practicas a la hora de definir un schema:
- Documentar los atributos y el esquema: esta información se envía al LLM y se usa para mejorar la calidad de la extracción de texto.
- No forzar al LLM a inventar información. Arriba se hace uso de "Optional" para los atributos permitiendo al LLM devolver None si este no sabe la respuesta.

# The extractor

Aquí se crea un extractor de información usando el schema definido.

In [3]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

Aquí se necesita un modelo que soporte funciones/tool calling.

In [4]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [5]:
structured_llm = llm.with_structured_output(schema=Person)

In [6]:
text = "Alan Smith is 6 feet tall and has blond hair."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Person(name='Alan Smith', hair_color='blond', height_in_meters='1.83')

Los LLM son generativos, por lo que pueden hacer cosas como extraer y ajustar los datos a la estructura deseada como se observa que aquí convierte '6 feet' a '1.83 meters'

# Multiple Entities

En la mayoría de los casos, se necesitaría extraer una lista de entidades en vez de solo una.

Esto se puede lograr usando Pydantic anidando modelos de schemas uno dentro de otro:

In [7]:
from typing import List, Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )


class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

In [8]:
structured_llm = llm.with_structured_output(schema=Data)
text = "My name is Jeff, my hair is black and i am 6 feet tall. Anna has the same color hair as me."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Data(people=[Person(name='Jeff', hair_color='black', height_in_meters='1.83'), Person(name='Anna', hair_color='black', height_in_meters=None)])

# Reference Examples

El comportamiento de los LLM pueden ser dirigidos usando few-shoot prompting. Para modelos de chat, esto puede tener forma de una secuencia de pares de entrada y respuestas, mostrando el comportamiento esperado.

In [9]:
messages = [
    {"role": "user", "content": "2 🦜 2"},
    {"role": "assistant", "content": "4"},
    {"role": "user", "content": "2 🦜 3"},
    {"role": "assistant", "content": "5"},
    {"role": "user", "content": "3 🦜 4"},
]

response = llm.invoke(messages)
print(response.content)

7


El structured output normalmente hace uso de tool-calling por debajo. Esto tipicamente envuelve la generación de AI messages que contienen tool calls, así como tool messages que contienen resultados de tool calls. ¿Cómo debería lucir una secuencia de mensajes en este caso?

Diferentes proveedores de modelos tienen sus propios requerimientos de secuancias de mensajes validos. Algunos aceptarían una (repetida) secuencia de mensajes de la forma:
* user message
* AI message con tool call
* Tool message con un resultado

Otros requieren un AI message final requiriendo algun tipo de respuesta.

LangChain incluye una función de utilidad too_example_to_message que genera una secuencia valida para la mayoría de los proveedores. Simplifica la generación de few-shoot examples estructurados, requiriendo solamente representaciones Pydantic para las herramientas correspondientes.

Se pueden convertir pares de input strings y objetos Pydantic deseados en una secuencia de mensajes que pueden ser entregados a un modelo de chat. Por debajo, LangChain va a estructurar la llamada de herramientas para formatdo requerido por el proveedor. 

In [10]:
from langchain_core.utils.function_calling import tool_example_to_messages

examples = [
    (
        "The ocean is vast and blue. It's more than 20,000 feet deep.",
        Data(people=[]),
    ),
    (
        "Fiona traveled far from France to Spain.",
        Data(people=[Person(name="Fiona", height_in_meters=None, hair_color=None)]),
    ),
]


messages = []

for txt, tool_call in examples:
    if tool_call.people:
        # This final message is optional for some providers
        ai_response = "Detected people."
    else:
        ai_response = "Detected no people."
    messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))

/var/folders/4p/dh8b653x7_vc017_hm_z3lk80000gn/T/ipykernel_88723/81328896.py:23: LangChainBetaWarning: The function `tool_example_to_messages` is in beta. It is actively being worked on, so the API may change.
  messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))


Al inspeccionar los resultados, vemos que estos dos ejemplos de pares generaron ocho mensajes:

In [11]:
for message in messages:
    message.pretty_print()

================================ Human Message =================================

The ocean is vast and blue. It's more than 20,000 feet deep.
================================== Ai Message ==================================
Tool Calls:
  Data (92bf221e-4dd5-4591-9e38-bc829ee7e962)
 Call ID: 92bf221e-4dd5-4591-9e38-bc829ee7e962
  Args:
    people: []
================================= Tool Message =================================

You have correctly called this tool.
================================== Ai Message ==================================

Detected no people.
================================ Human Message =================================

Fiona traveled far from France to Spain.
================================== Ai Message ==================================
Tool Calls:
  Data (5f6fae57-0e38-4985-8e9d-a9424d860982)
 Call ID: 5f6fae57-0e38-4985-8e9d-a9424d860982
  Args:
    people: [{'name': 'Fiona', 'hair_color': None, 'height_in_meters': None}]
================================

Vamos a comparar el rendimiento con y sin estos mensajes. Por ejemplo, vamos a pasar un mensaje donde esperamos que ninguna información de "persona" sea extraido.

In [12]:
message_no_extraction = {
    "role": "user",
    "content": "The solar system is large, but earth has only 1 moon.",
}

structured_llm = llm.with_structured_output(schema=Data)
structured_llm.invoke([message_no_extraction])

Data(people=[])

En la documentación se muestra que el modelo incorrectamente clasifica la información del texto de ejemplo. Pero en nuestra prueba vemos que no funciona, esto podría deberse porque se ha mejorado el rendimiento del modelo.

Al pasar los mensajes, se esperaría mejorar el rendimiento de la extracción:

In [13]:
structured_llm.invoke(messages + [message_no_extraction])

Data(people=[])